In [1]:
import os
import scipy.io as sio
import numpy as np

# NB

In [2]:
NB_keys = ['ground', 'low_vegetation', 'medium_vegetation', 'high_vegetation', 'building', 'others']

In [3]:
def get_classwise_RMSE (keys, seg, pred, gt) :
    label_dics = {'ground': 2, 
              'low_vegetation': 3, 
              'medium_vegetation': 4,
              'high_vegetation': 5,
              'building': 6,
              'others': 0}

    total_num = gt.shape[0] * gt.shape[1]
    
    others_index = np.ones_like(seg)
    rmse_dict = {}
    occup_dict = {}
    
    RMSE = np.sqrt(np.mean((pred - gt)**2))
    rmse_dict['total'] = np.round(RMSE, 2)
    
    # class-wise results
    for key in keys :
        #if key == 'ground' :
        #    mask = np.where((seg == 2) | (seg==8))
        #    num = mask[0].shape[0]
        #    others_index[mask] = 0
        if key == 'others' :
            mask = np.where(others_index == 1)
            num = mask[0].shape[0]
        else :
            label = label_dics[key]
            mask = np.where(seg == label)
            num = mask[0].shape[0]
            others_index[mask] = 0
        if not num == 0 :
            RMSE_cls = np.sqrt(np.mean((pred[mask] - gt[mask])**2))
        else :
            RMSE_cls = 0
        
        rmse_dict[key] = np.round(RMSE_cls, 2)
        occup_dict[key] = np.round(100*num/total_num, 2)
        
    return rmse_dict, occup_dict
        

In [7]:
# NB
with open('/nas2/YJ/DATA/ALS2DTM/splits/NB-test.tiles', 'r') as f :
    lines = f.readlines()

MAT_ROOT = '/nas2/YJ/DATA/ALS2DTM/NB'
PRED_ROOT = '/nas2/YJ/git/DeepTerRa_pix2pix/results/NB_Elevs_b32_lr0.01_e300_700/'

RMSE = 0

with open(os.path.join(PRED_ROOT, 'classwise_resuts_NB_elevs.csv'), 'w') as f : 
    f.write('filename,total,ground,low_vegetation,medium_vegetation,high_vegetation,building,others,ground,low_vegetation,medium_vegetation,high_vegetation,building,others\n')
    for line in lines :
        str1, str2 = line.strip().split('_')
        matfilename = 'nb_2018_{}00_{}00.mat'.format(str1, str2)

        matfile = os.path.join(MAT_ROOT, matfilename)
        mat = sio.loadmat(matfile)
        dtm = mat['dtm']
        seg = mat['semantics']

        h, w = dtm.shape

        left = (512 - h) //2
        top = (512 - w) //2

        npyfile = os.path.join(PRED_ROOT, matfilename.replace('mat', 'npy'))
        pred = np.load(npyfile) [left:left+h, top:top+w]

        rmse_dict, occup_dict = get_classwise_RMSE (NB_keys, seg, pred, dtm)
        
        msg = matfilename + ',' + str(rmse_dict['total'])
        for key in NB_keys : 
            msg+= ',' + str(rmse_dict[key])
        for key in NB_keys :
            msg += ',' + str(occup_dict[key])
        msg += '\n'
        f.write(msg)
        #print (matfilename, rmse_dict, occup_dict)

# DALES
- ground(1), vegetation(2), cars(3), trucks(4), power lines(5), fences(6), poles(7) and buildings(8).

In [2]:
DALES_keys = ['ground', 'vegetation', 'cars', 'trucks', 'power_lines', 'fences', 'poles', 'buildings', 'others']

In [3]:
def get_classwise_RMSE_DALES (keys, seg, pred, gt) :
    label_dics = {'ground': 1, 
                  'vegetation': 2,
                  'cars': 3,
                  'trucks': 4,
                  'power_lines': 5,
                  'fences': 6,
                  'poles': 7,
                  'buildings': 8,
                  'others': 0}

    total_num = gt.shape[0] * gt.shape[1]
    
    others_index = np.ones_like(seg)
    rmse_dict = {}
    occup_dict = {}
    
    RMSE = np.sqrt(np.mean((pred - gt)**2))
    rmse_dict['total'] = np.round(RMSE, 2)
    
    # class-wise results
    for key in keys :
        #if key == 'ground' :
        #    mask = np.where((seg == 2) | (seg==8))
        #    num = mask[0].shape[0]
        #    others_index[mask] = 0
        #if key == 'others' :
        #    mask = np.where(others_index == 1)
        #    num = mask[0].shape[0]
        #else :
        label = label_dics[key]
        mask = np.where(seg == label)
        num = mask[0].shape[0]
        others_index[mask] = 0
        if not num == 0 :
            RMSE_cls = np.sqrt(np.mean((pred[mask] - gt[mask])**2))
        else :
            RMSE_cls = 0
        
        rmse_dict[key] = np.round(RMSE_cls, 2)
        occup_dict[key] = np.round(100*num/total_num, 2)
        
    return rmse_dict, occup_dict
        

In [7]:
with open('/nas2/YJ/DATA/ALS2DTM/splits/DALES-test.tiles', 'r') as f :
    lines = f.readlines()

NPY_ROOT = '/nas2/YJ/DATA/ALS2DTM/DALESDSP/test/'
PRED_ROOT = '/nas2/YJ/git/DeepTerRa_pix2pix/results/DALESDSP_Mean_b32_lr0.01_e300_700/'

#rasters = [ 'voxel-top',  'voxel-bottom', 'pixel-mean', 'density', 'stdev', 'echoes']

with open(os.path.join(PRED_ROOT, 'classwise_resuts_dales_downsampling_Mean.csv'), 'w') as f : 
    header = 'filename, total'
    for key in DALES_keys :
        header += ',' + key
    for key in DALES_keys :
        header += ',' + key
    header += '\n'
    f.write(header)
    
    for line in lines :
        for i in range(4) :
            for j in range(4) :
                str1, str2 = line.strip().split('_')
                npyfilename = '{}_{}_{}_{}.npy'.format(str1, str2, i, j)
                dtm = np.load(os.path.join(NPY_ROOT, 'dtm', npyfilename))
                seg = np.load(os.path.join(NPY_ROOT, 'semantics', npyfilename))
                
                h, w = dtm.shape

                left = (512 - h) //2
                top = (512 - w) //2

                npyfile = os.path.join(PRED_ROOT, npyfilename)
                pred = np.load(npyfile) [left:left+h, top:top+w]

                rmse_dict, occup_dict = get_classwise_RMSE_DALES(DALES_keys, seg, pred, dtm)

                msg = npyfilename + ',' + str(rmse_dict['total'])
                for key in DALES_keys : 
                    msg+= ',' + str(rmse_dict[key])
                for key in DALES_keys :
                    msg += ',' + str(occup_dict[key])
                msg += '\n'
                f.write(msg)

In [70]:
def get_rmse(dtm, pred) :

    h, w = dtm.shape
    
    diff_x = 512 - h
    diff_y = 512 - w
    left = diff_x // 2
    top = diff_y // 2
    
    results = np.zeros((left+1, top+1))
    for i in range (left+1) :
        for j in range (top +1) :
            results[i, j] = np.sqrt(np.mean((dtm - pred[i:i+h, j : j+w])**2))
    
    print (np.unravel_index(np.argmin(results, axis=None), results.shape), (left, top), results.min(), results[left, top])
    return results[left, top]

In [76]:
with open('/nas2/YJ/DATA/ALS2DTM/splits/NB-test.tiles', 'r') as f :
    lines = f.readlines()

MAT_ROOT = '/nas2/YJ/DATA/ALS2DTM/NB'
PRED_ROOT = '/nas2/YJ/git/DeepTerRa_pix2pix/results/bot-b32lr0.01e300-700/'

RMSE = 0
for line in lines :
    str1, str2 = line.strip().split('_')
    matfilename = 'nb_2018_{}00_{}00.mat'.format(str1, str2)
    
    matfile = os.path.join(MAT_ROOT, matfilename)
    mat = sio.loadmat(matfile)
    dtm = mat['dtm']
    
    npyfile = os.path.join(PRED_ROOT, matfilename.replace('mat', 'npy'))
    pred = np.load(npyfile)
    
    RMSE += get_rmse(dtm, pred)

(6, 6) (7, 6) 2.546346664428711 2.551379919052124
(7, 6) (7, 6) 0.19417621195316315 0.19417621195316315
(7, 6) (7, 6) 1.1636289358139038 1.1636289358139038
(7, 6) (7, 6) 0.2883918285369873 0.2883918285369873
(7, 6) (7, 6) 0.8836695551872253 0.8836695551872253
(6, 6) (7, 6) 0.3329331576824188 0.33378177881240845
(7, 6) (7, 6) 0.9864586591720581 0.9864586591720581
(7, 6) (7, 6) 0.8558001518249512 0.8558001518249512
(7, 6) (7, 6) 0.3416290879249573 0.3416290879249573
(7, 6) (7, 6) 0.18919116258621216 0.18919116258621216
(6, 5) (7, 6) 1.1255500316619873 1.1322085857391357
(7, 6) (7, 6) 0.28124091029167175 0.28124091029167175
(7, 6) (7, 6) 0.6760918498039246 0.6760918498039246
(7, 6) (7, 6) 0.34453511238098145 0.34453511238098145
(7, 6) (7, 6) 0.4135798513889313 0.4135798513889313
(7, 6) (7, 6) 0.24764840304851532 0.24764840304851532
(7, 6) (7, 6) 0.159226194024086 0.159226194024086
(7, 6) (7, 6) 0.23307383060455322 0.23307383060455322
(7, 6) (7, 6) 1.1979714632034302 1.1979714632034302
(7,

In [77]:
RMSE / len(lines)

0.6658405732540857

nb_2018_2537000_7369000.mat {'total': [2.55138, 248502], 'ground': [1.2538941, 0.36524454531553063], 'low_vegetation': [1.4484653, 0.04894125600598788], 'medium_vegetation': [1.2476892, 0.07455070784138558], 'high_vegetation': [3.3655488, 0.4328737796878898], 'building': [3.4824147, 0.06433751036208964], 'others': [1.5367869, 0.0140522007871164]}
nb_2018_2613000_7402500.mat {'total': [0.19417623, 248502], 'ground': [0.10363722, 0.0003500977859333124], 'low_vegetation': [0.13493116, 0.0009698111081600952], 'medium_vegetation': [0.12800992, 0.0009939557830520478], 'high_vegetation': [0.1943046, 0.9976780870978905], 'building': [0, 0.0], 'others': [0.10499271, 8.048224963984193e-06]}
nb_2018_2538000_7367000.mat {'total': [1.1636289, 248502], 'ground': [0.5568881, 0.1860991058422065], 'low_vegetation': [0.5497984, 0.15850576655318668], 'medium_vegetation': [0.91163963, 0.10018832846415723], 'high_vegetation': [1.3458325, 0.5379192119178116], 'building': [3.340525, 0.017098453935984418], 'o

In [63]:
mat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'dtm', 'voxel-top', 'semantics', 'voxel-bottom', 'pixel-mean', 'density', 'stdev', 'echoes'])

In [26]:
pred[left:left+h, top: top+w].shape

(498, 499)

(0, 2) 2.6039645671844482 2.766711950302124


In [51]:
np.where(results == results.min())

(array([0]), array([2]))

In [50]:
results.min()

2.6039645671844482

In [52]:
results.argmin()

2

(0, 2)

In [ ]:
GT_PATH = '/nas2/YJ/DATA/ALS2DTM/NB/Top/test/dtm'
SEG_PATH = '/nas2/YJ/DATA/ALS2DTM/NB/Top/test/seg'
PRED_PATH = '/nas2/YJ/git/ALS2DTM/results/top_tanh_crop_flip_b32_lr0.01_e1000//'

filenames = [f for f in os.listdir(GT_PATH) if 'npy' in f]
keys = ['ground', 'low_vegetation', 'medium_vegetation', 'high_vegetation', 'building', 'others']

for filename in filenames[:10] :
    #mat = sio.loadmat(os.path.join(NB_ROOT, filename))
    seg = np.load(os.path.join(SEG_PATH, filename))[0,6:-6,6:-6]
    
    gt = np.load(os.path.join(GT_PATH, filename))[0,6:-6,6:-6]
    pred = np.load(os.path.join(PRED_PATH, filename))[0,6:-6,6:-6]
    
    result_dict = get_RMSE(keys, seg, pred, gt)
    print (filename, result_dict)